In [3]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.decomposition import NMF

import re # The following statement imports the regex package.
import gensim # The following statement imports the gensim package.
import nltk # The following statement imports the NLTK package.
import pkg_resources
import pandas as pd
import os
import numpy as np
#from symspellpy import SymSpell, Verbosity
from nltk import pos_tag, word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)
%cd gdrive/MyDrive/TxtAnalytics_Proj

Mounted at /content/gdrive
/content/gdrive/MyDrive/TxtAnalytics_Proj


In [ ]:
def generateVectors(posts, filename):
         #create dictionary
        all_docs5 = []

        suicidal_data_TFIDF = list()
        suicidal_data_TF = list()

        #create corpus dictionary
        corpus_dictionary = gensim.corpora.Dictionary()

        stop_list = ("suicidal", "suicide", "aah", "fuck")
        
        
        for i in range(len(posts)):
            value = posts.iloc[i, posts.columns.get_loc("text_lemmatized")]
            #print(value)
            all_docs5.append(value) #df.iloc[0, df.columns.get_loc("a")]

        all_docs5 = [[w for w in doc if w not in stop_list] for doc in all_docs5]
               
        corpus_dictionary.add_documents(all_docs5)
        
        # Convert all documents to term frequency (TF) vectors
        all_tf_vectors = [corpus_dictionary.doc2bow(doc) for doc in all_docs5]

        #ntc = n = raw, t = zero-corrected idf, c = cosine - https://radimrehurek.com/gensim/models/tfidfmodel.html
        tfidf = gensim.models.TfidfModel(all_tf_vectors, smartirs='ntc')
        corpus_tfidf = tfidf[all_tf_vectors]
        
        all_data_as_dict = [{id:tf_value for (id, tf_value) in vec} for vec in all_tf_vectors]
        tfidf_data_as_dict = [{id:tf_value for (id, tf_value) in vec} for vec in corpus_tfidf]
       
        for i in range(len(all_data_as_dict)):
            doc_tf = all_data_as_dict[i]
            doc_tfidf = tfidf_data_as_dict[i]

            suicidal_data_TF.append((doc_tf, 1))
            suicidal_data_TFIDF.append((doc_tfidf, 1))

      
        #print(suicidal_data_TFIDF)
        posts['tf_vector'] = suicidal_data_TF 
        posts['tfidf_vector']  = suicidal_data_TFIDF

        posts.to_pickle(filename)

        corpus_dictionary.save(filename.replace(".pkl", ".dict"))

        return posts

In [2]:
#data_raw =  pd.read_pickle("posts_after_lemmatized.pkl")
!pip install pandas==1.5.3
print('The pandas version is {}.'.format(pd.__version__))
#df_filtered =  pd.read_pickle("posts_suicidal_with_tfidf2.pkl")
#df_filtered = generateVectors(df_filtered, "posts_suicidal_with_tfidf2.pkl")
#df_filtered.head()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The pandas version is 1.5.3.


In [4]:


!pip install fast_ml

import fast_ml
from fast_ml.model_development import train_valid_test_split
X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(df_filtered, target = 'class', 
                                                                            train_size=0.7, valid_size=0.2, test_size=0.1)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 KB 2.7 MB/s eta 0:00:00


NameError: ignored

In [1]:
X_train.to_pickle("NMF_X_Train.pkl")
X_test.to_pickle("NMF_X_Test.pkl")
X_valid.to_pickle("NMF_X_Valid.pkl")

NameError: ignored

In [1]:
from sklearn.decomposition import NMF
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV, GridSearchCV
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_score
from sklearn.feature_extraction.text import TfidfVectorizer

# Define custom scoring function
def my_score(X, labels):
    return silhouette_score(X, labels=True)

from sklearn.metrics import mean_squared_error, mean_squared_log_error


In [5]:
!pip install pandas==1.5.3
import pandas as pd
#print('The pandas version is {}.'.format(pd.__version__))

vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
trainingData = pd.read_pickle("NMF_X_Train.pkl")



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
post_split_Train_X = np.array_split(trainingData, 5)
searchMatrix = post_split_Train_X[0]
stop_list = ("suicidal", "suicide", "aah", "fuck", "fucking")
text_data = searchMatrix['text_lemmatized'].to_numpy()
text_data = [[w for w in doc if w not in stop_list] for doc in text_data]

text_data_lemmatized_joined = [' '.join(x) for x in text_data]  # joined to fit CountVectorizer

vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')

# Fit the vectorizer and transform the text data
X = vectorizer.fit_transform(text_data_lemmatized_joined)

param_grid = {'l1_ratio': [0, 0.2, 0.25, 0.5, 1],
               'alpha_H': [0, 0.01, 0.001],
               'alpha_W': [0, 0.01, 0.001],
              }

nmf = NMF(alpha_H= 0.001, random_state=10,
                n_components=10
                ,init = 'random'
                )
x_2d = X.reshape(-1, 1)
grid_search = HalvingGridSearchCV(estimator=nmf, param_grid=param_grid, scoring=my_score,
                                  aggressive_elimination = True ,
                                  cv=5)

grid_search.fit(x_2d)


In [ ]:
def GridSearch_table_plot(grid_clf, param_name,
                          num_results=15,
                          negative=True,
                          graph=True,
                          display_all_params=True):

    '''Display grid search results

    Arguments
    ---------

    grid_clf           the estimator resulting from a grid search
                       for example: grid_clf = GridSearchCV( ...

    param_name         a string with the name of the parameter being tested

    num_results        an integer indicating the number of results to display
                       Default: 15

    negative           boolean: should the sign of the score be reversed?
                       scoring = 'neg_log_loss', for instance
                       Default: True

    graph              boolean: should a graph be produced?
                       non-numeric parameters (True/False, None) don't graph well
                       Default: True

    display_all_params boolean: should we print out all of the parameters, not just the ones searched for?
                       Default: True

    Usage
    -----

    GridSearch_table_plot(grid_clf, "min_samples_leaf")

                          '''
    from matplotlib      import pyplot as plt
    from IPython.display import display
    import pandas as pd

    clf = grid_clf.best_estimator_
    clf_params = grid_clf.best_params_
    if negative:
        clf_score = -grid_clf.best_score_
    else:
        clf_score = grid_clf.best_score_
    clf_stdev = grid_clf.cv_results_['std_test_score'][grid_clf.best_index_]
    cv_results = grid_clf.cv_results_

    print("best parameters: {}".format(clf_params))
    print("best score:      {:0.10f} (+/-{:0.10f})".format(clf_score, clf_stdev))
    if display_all_params:
        import pprint
        pprint.pprint(clf.get_params())

    # pick out the best results
    # =========================
    scores_df = pd.DataFrame(cv_results).sort_values(by='rank_test_score')
    #print(scores_df)
    best_row = scores_df.iloc[0, :]
    if negative:
        best_mean = -best_row['mean_test_score']
    else:
        best_mean = best_row['mean_test_score']
    best_stdev = best_row['std_test_score']
    best_param = best_row['param_' + param_name]

    # display the top 'num_results' results
    # =====================================
    display(pd.DataFrame(cv_results) \
            .sort_values(by='rank_test_score').head(num_results))

    # plot the results
    # ================
    scores_df = scores_df.sort_values(by='param_' + param_name)

    if negative:
        means = -scores_df['mean_test_score']
    else:
        means = scores_df['mean_test_score']
    stds = scores_df['std_test_score']
    params = scores_df['param_' + param_name]

    # plot
    if graph:
        plt.figure(figsize=(8, 8))
        plt.errorbar(params, means, yerr=stds)

        plt.axhline(y=best_mean + best_stdev, color='red')
        plt.axhline(y=best_mean - best_stdev, color='red')
        plt.plot(best_param, best_mean, 'or')

        plt.title(param_name + " vs Score\nBest Score {:0.10f}".format(clf_score))
        plt.xlabel(param_name)
        plt.ylabel('Score')
        plt.show()


# Print best parameters
print(grid_search.best_params_)

In [63]:
searchMatrix = pd.read_pickle("NMF_X_Train.pkl")
stop_list = ("suicidal", "suicide", "aah", "fuck", "fucking")
text_data = searchMatrix['text_lemmatized'].to_numpy()
text_data = [[w for w in doc if w not in stop_list] for doc in text_data]

text_data_lemmatized_joined = [' '.join(x) for x in text_data]  # joined to fit CountVectorizer

vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
# Fit the vectorizer and transform the text data
data_vectorized = vectorizer.fit_transform(text_data_lemmatized_joined)

feature_names1 = vectorizer.get_feature_names_out()


In [64]:

NMF_model = NMF(alpha_H= 0.001, 
                #alpha_W= 0.001, 
                l1_ratio=0.25, 
                n_components=10
                ,init = 'nndsvd'
                )

nmf_output = NMF_model.fit_transform(data_vectorized)
print(nmf_output)  # Model attributes

[[ 0.          0.          0.         ...  4.41256346 10.89402751
   0.        ]
 [ 7.96657627  2.93622238  1.19345597 ...  2.42184209  3.94265621
   3.56154354]
 [14.08958421  1.07652326  0.         ...  4.17180488  0.55579553
   2.84125974]
 ...
 [13.38826749  0.07772303  1.66232003 ...  3.4688774   0.33474805
   0.        ]
 [14.56718567  1.62035948  0.5999533  ...  0.          0.14860755
   1.692225  ]
 [ 5.3709157   0.46356567  0.91485681 ...  1.48415679  2.11658345
   0.70396175]]


/usr/local/lib/python3.8/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [81]:

NMF_model2 = NMF(alpha_H= 0.001, 
                #alpha_W= 0.001, 
                l1_ratio=0.4, 
                n_components=10
                ,init = 'random'
                )
nmf_output2 = NMF_model2.fit_transform(data_vectorized)
print(nmf_output2)  # Model attributes

[[ 0.          0.          0.         ...  5.81002297 15.75114063
   0.        ]
 [12.73631359  3.7283255   1.50104599 ...  3.02307932  5.73646778
   5.69806346]
 [21.46166368  1.37763077  0.         ...  5.24802081  0.74717588
   4.52971796]
 ...
 [20.55231362  0.09204465  2.1172661  ...  4.43590157  0.51129029
   0.        ]
 [21.9155353   2.06141005  0.75427319 ...  0.          0.23256519
   2.65845015]
 [ 8.34499944  0.58903731  1.19882685 ...  1.86153893  3.04656771
   1.15779609]]


/usr/local/lib/python3.8/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [80]:
#print(data_vectorized[:20][0:])
# Print the top words for each topic
feature_names = vectorizer.get_feature_names_out()
#for topic_idx, topic in enumerate(NMF_model.components_):
#    top_words = [feature_names[i] for i in topic.argsort()[:-15 - 1:-1]]
#    print(f"Topic {topic_idx}: {', '.join(top_words)}")

for topic_idx, topic in enumerate(NMF_model2.components_):
    top_words = [feature_names[i] for i in topic.argsort()[:-15 - 1:-1]]
    print(f"Topic {topic_idx}: {', '.join(top_words)}")



Topic 0: time, year, day, friend, think, school, tell, work, start, know, didst, thing, try, month, say
Topic 1: want, die, end, stop, pain, hurt, scar, sleep, wish, wake, badly, tire, dead, happy, death
Topic 2: feel, like, feeling, bad, know, make, thought, felt, way, right, think, sad, time, day, happy
Topic 3: help, need, know, try, post, advice, ask, tell, thought, therapy, seek, sure, therapist, thank, wont
Topic 4: na, wan, gon, die, shit, end, soon, tonight, probably, ill, cut, tomorrow, bad, id, hang
Topic 5: live, life, end, point, year, reason, worth, pain, lose, job, try, continue, happy, ill, family
Topic 6: people, hate, care, know, love, wish, world, good, life, person, think, hurt, thing, nt, shit
Topic 7: kill, think, tonight, reason, ill, try, self, plan, right, way, today, wish, dead, scar, soon
Topic 8: anymore, tire, know, hurt, tired, pain, care, handle, try, point, goodbye, deal, tonight, shit, alive
Topic 9: talk, need, friend, tell, say, lonely, right, want, me

In [ ]:
import joblib

# Save a model to disk
joblib.dump(NMF_model, 'sg_nmf.jl')
joblib.dump(nmf_output, 'sg_nmf_output.jl')

In [23]:
!pip install pyldavis pyLDAvis==3.4.0
import pyLDAvis
import pyLDAvis.lda_model
pyLDAvis.enable_notebook()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
print('The pyLDAvis-learn version is {}.'.format(pyLDAvis.__version__))
panel = pyLDAvis.lda_model.prepare(NMF_model, data_vectorized, vectorizer, mds='tsne')
panel

The pyLDAvis-learn version is 3.4.0.


/usr/local/lib/python3.8/dist-packages/pyLDAvis/lda_model.py:26: RuntimeWarning: invalid value encountered in true_divide
  return dists / dists.sum(axis=1)[:, None]


ValueError: ignored